# ISS

In [ ]:
from ipyrest import Api

url = 'https://api.wheretheiss.at/v1/satellites/25544'
Api(url)

In [ ]:
import json

import requests
from ipywidgets import Textarea, Layout
from ipyrest import Api
from ipyrest.responseviews import ResponseView

class IssAddressView(ResponseView):
    name = 'ISSAddress'
    mimetype_pats = ['application/json']
    def render(self, resp):
        j_iss = resp.json()
        lat, lon = j_iss['latitude'], j_iss['longitude']
        url = 'https://nominatim.openstreetmap.org/reverse'\
              f'?format=json&lat={lat}&lon={lon}&addressdetails=1'
        j_osm = requests.get(url).json()
        j_iss['addr'] = "Error: 'Unable to geocode'" \
            if j_osm == {'error': 'Unable to geocode'} \
            else j_osm['display_name']
        layout = Layout(width='100%', height='100%')
        return Textarea(value=json.dumps(j_iss, indent=4), layout=layout)

url = 'https://api.wheretheiss.at/v1/satellites/25544'
Api(url, additional_views=[IssAddressView])

In [ ]:
# from http://www.ustream.tv/channel/iss-hdev-payload
from IPython.display import IFrame
IFrame(src="http://www.ustream.tv/embed/17074538?html5ui", width="100%", height="600")

In [ ]:
from ipyleaflet import Map, Marker

class IssMapView(ResponseView):
    name = 'ISSMap'
    mimetype_pats = ['application/json']
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.m = Map()
        return
    
        # nicer with: http://polymaps.org/ex/blue-marble.html
        url = "http://s3.amazonaws.com/com.modestmaps.bluemarble/{z}-r{y}-c{x}.jpg"
        attr = '<a href="https://earthobservatory.nasa.gov/">NASA Earth Observatory</a>'
        bm = dict(url=url, attribution=attr)
        self.m = Map(zoom=6, basemap=bm)

    def render(self, resp):
        j_iss = resp.json()
        lat, lon = j_iss['latitude'], j_iss['longitude']
        self.m.center=(lat, lon)
        self.m += Marker(location=(lat, lon))
        return self.m

url = 'https://api.wheretheiss.at/v1/satellites/25544'
Api(url, additional_views=[IssAddressView, IssMapView])